# Smoothing_Quick: MediaPipe Built-in Smoothing Only

This simple notebook extracts MediaPipe keypoints from a video using MediaPipe's built-in smoothing (enabled when `static_image_mode=False`). It saves time series CSV files for body, hands, and face.

- Recommended for most use cases where smoothed keypoint positions are sufficient
- For smoothing derivatives (speed, acceleration, jerk), use the full `Smoothing.ipynb` pipeline
- Looking for the full signal-processing suite? See `Smoothing.ipynb` (Butterworth, Savitzky–Golay, Gaussian, and evaluation).


In [ ]:
# Imports and Paths
import os
import csv
import cv2
import mediapipe as mp
import numpy as np
import shutil

# Configure folders (relative to this notebook)
input_folder = "../data/interim"  # place your .mp4 videos here
output_folder = "../results/Output_TimeSeries"

# Option: use existing CSVs from MediaPipe_keypoints_extraction
use_existing_csvs = False
input_csv_folder = "../Mediapipe_results"  # where *_body.csv, *_hands.csv, *_face.csv are located

os.makedirs(output_folder, exist_ok=True)

# List videos
video_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".mp4", ".mov", ".mkv"))]
print("Videos:", video_files)



In [ ]:
# Landmark column headers
mp_holistic = mp.solutions.holistic

markers_body = [
    'NOSE','LEFT_EYE_INNER','LEFT_EYE','LEFT_EYE_OUTER','RIGHT_EYE_INNER','RIGHT_EYE','RIGHT_EYE_OUTER',
    'LEFT_EAR','RIGHT_EAR','MOUTH_LEFT','MOUTH_RIGHT','LEFT_SHOULDER','RIGHT_SHOULDER','LEFT_ELBOW',
    'RIGHT_ELBOW','LEFT_WRIST','RIGHT_WRIST','LEFT_PINKY','RIGHT_PINKY','LEFT_INDEX','RIGHT_INDEX',
    'LEFT_THUMB','RIGHT_THUMB','LEFT_HIP','RIGHT_HIP','LEFT_KNEE','RIGHT_KNEE','LEFT_ANKLE','RIGHT_ANKLE',
    'LEFT_HEEL','RIGHT_HEEL','LEFT_FOOT_INDEX','RIGHT_FOOT_INDEX']

markers_hands = [
    'LEFT_WRIST','LEFT_THUMB_CMC','LEFT_THUMB_MCP','LEFT_THUMB_IP','LEFT_THUMB_TIP','LEFT_INDEX_FINGER_MCP',
    'LEFT_INDEX_FINGER_PIP','LEFT_INDEX_FINGER_DIP','LEFT_INDEX_FINGER_TIP','LEFT_MIDDLE_FINGER_MCP',
    'LEFT_MIDDLE_FINGER_PIP','LEFT_MIDDLE_FINGER_DIP','LEFT_MIDDLE_FINGER_TIP','LEFT_RING_FINGER_MCP',
    'LEFT_RING_FINGER_PIP','LEFT_RING_FINGER_DIP','LEFT_RING_FINGER_TIP','LEFT_PINKY_FINGER_MCP',
    'LEFT_PINKY_FINGER_PIP','LEFT_PINKY_FINGER_DIP','LEFT_PINKY_FINGER_TIP',
    'RIGHT_WRIST','RIGHT_THUMB_CMC','RIGHT_THUMB_MCP','RIGHT_THUMB_IP','RIGHT_THUMB_TIP','RIGHT_INDEX_FINGER_MCP',
    'RIGHT_INDEX_FINGER_PIP','RIGHT_INDEX_FINGER_DIP','RIGHT_INDEX_FINGER_TIP','RIGHT_MIDDLE_FINGER_MCP',
    'RIGHT_MIDDLE_FINGER_PIP','RIGHT_MIDDLE_FINGER_DIP','RIGHT_MIDDLE_FINGER_TIP','RIGHT_RING_FINGER_MCP',
    'RIGHT_RING_FINGER_PIP','RIGHT_RING_FINGER_DIP','RIGHT_RING_FINGER_TIP','RIGHT_PINKY_FINGER_MCP',
    'RIGHT_PINKY_FINGER_PIP','RIGHT_PINKY_FINGER_DIP','RIGHT_PINKY_FINGER_TIP']

markers_face = [str(x) for x in range(478)]

columns_body = ['time'] + [f"{pos}_{m}" for m in markers_body for pos in ['X','Y','Z','visibility']]
columns_hands = ['time'] + [f"{pos}_{m}" for m in markers_hands for pos in ['X','Y','Z']]
columns_face = ['time'] + [f"{pos}_{m}" for m in markers_face for pos in ['X','Y','Z']]



In [ ]:
# Process videos with MediaPipe built-in smoothing or reuse existing CSVs
for video_file in video_files:
    base = os.path.splitext(video_file)[0]
    video_path = os.path.join(input_folder, video_file)

    if use_existing_csvs:
        # Copy existing CSVs if present
        for suffix in ["_body.csv", "_hands.csv", "_face.csv"]:
            src = os.path.join(input_csv_folder, base + suffix)
            dst = os.path.join(output_folder, base + suffix)
            if os.path.exists(src):
                shutil.copy2(src, dst)
                print(f"Copied existing: {os.path.basename(src)}")
            else:
                print(f"Missing expected file: {os.path.basename(src)}")
        continue

    print(f"Processing: {video_file}")

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

    # Initialize time series containers
    ts_body = [columns_body]
    ts_hands = [columns_hands]
    ts_face = [columns_face]
    time_ms = 0.0

    with mp_holistic.Holistic(
        static_image_mode=False,   # enables MediaPipe built-in smoothing
        model_complexity=1,
        enable_segmentation=False,
        refine_face_landmarks=True
    ) as holistic:
        while True:
            ok, frame = cap.read()
            if not ok:
                break
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image_rgb)

            # Body
            if results.pose_landmarks is not None:
                body_vals = []
                for lm in results.pose_landmarks.landmark:
                    body_vals.extend([lm.x, lm.y, lm.z, lm.visibility])
            else:
                body_vals = [np.nan] * (len(columns_body) - 1)

            # Hands (left + right)
            def hand_vals(landmarks):
                if landmarks is None:
                    return [np.nan] * (21 * 3)
                vals = []
                for lm in landmarks.landmark:
                    vals.extend([lm.x, lm.y, lm.z])
                return vals

            left_hand = hand_vals(results.left_hand_landmarks)
            right_hand = hand_vals(results.right_hand_landmarks)
            hands_vals = left_hand + right_hand

            # Face
            if results.face_landmarks is not None:
                face_vals = []
                for lm in results.face_landmarks.landmark:
                    face_vals.extend([lm.x, lm.y, lm.z])
            else:
                face_vals = [np.nan] * (len(columns_face) - 1)

            # Insert time and append rows
            row_body = [time_ms] + body_vals
            row_hands = [time_ms] + hands_vals
            row_face = [time_ms] + face_vals

            ts_body.append(row_body)
            ts_hands.append(row_hands)
            ts_face.append(row_face)

            time_ms += (1000.0 / fps)

    cap.release()

    # Save CSVs
    with open(os.path.join(output_folder, f"{base}_body.csv"), "w", newline="") as f:
        csv.writer(f).writerows(ts_body)
    with open(os.path.join(output_folder, f"{base}_hands.csv"), "w", newline="") as f:
        csv.writer(f).writerows(ts_hands)
    with open(os.path.join(output_folder, f"{base}_face.csv"), "w", newline="") as f:
        csv.writer(f).writerows(ts_face)

    print(f"Saved: {base}_body.csv, {base}_hands.csv, {base}_face.csv")



## Inspect Video (Optional)

Play the input video to visually inspect the content before/after processing.


In [ ]:
from IPython.display import Video, display

# Preview the first video (if present)
if len(video_files) > 0:
    preview_path = os.path.join(input_folder, video_files[0])
    print("Previewing:", video_files[0])
    display(Video(preview_path, embed=True, width=640))
else:
    print("No videos found in:", input_folder)



### Done

CSV files have been saved to `../results/Output_TimeSeries/`. Use these files for downstream smoothing of derivatives (speed, acceleration, jerk) or other analyses.
